In [5]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

In [2]:
import cv2

# Verify the OpenCV version
print(cv2.__version__)

# Test OpenCV functionality
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    ret, frame = cap.read()
    if ret:
        cv2.imshow('Test Frame', frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    cap.release()


4.10.0


In [16]:
import cv2
import mediapipe as mp
import numpy as np
import os
import json
import math

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
    return []

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []
    angles_list = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        keypoints = extract_keypoints(frame)
        if keypoints:
            keypoints_list.append(keypoints)
            # Calculate angles (example: elbow angle)
            angle = calculate_angle(keypoints[11], keypoints[13], keypoints[15])  # Example for left elbow
            angles_list.append(angle)

    cap.release()
    return keypoints_list, angles_list

def process_dataset(dataset_path, output_path):
    exercises = ['bench press', 'squat', 'deadlift']
    all_data = []

    for exercise in exercises:
        exercise_path = os.path.join(dataset_path, exercise)
        for video_file in os.listdir(exercise_path):
            if video_file.endswith('.mp4'):
                video_path = os.path.join(exercise_path, video_file)
                keypoints, angles = process_video(video_path)
                label = exercise
                all_data.append({'keypoints': keypoints, 'angles': angles, 'label': label})
    
    with open(output_path, 'w') as f:
        json.dump(all_data, f)
    print(f'Saved keypoints to {output_path}')

if __name__ == '__main__':
    dataset_path = 'sbd_dataset'  # Change to your dataset path
    output_path = 'keypoints_data.json'
    process_dataset(dataset_path, output_path)


C:\Users\ramzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Saved keypoints to keypoints_data.json


In [17]:
import json
import numpy as np
from sklearn.model_selection import train_test_split

# Load the extracted keypoints data
with open('keypoints_data.json', 'r') as f:
    data = json.load(f)

# Flatten keypoints and create labels
X = []
y = []
for item in data:
    keypoints = item['keypoints']
    angles = item['angles']
    label = item['label']
    for frame_keypoints, angle in zip(keypoints, angles):
        features = np.array(frame_keypoints).flatten().tolist() + [angle]
        X.append(features)
        y.append(label)

# Convert labels to numerical values
label_to_num = {'bench press': 0, 'squat': 1, 'deadlift': 2}
y = [label_to_num[label] for label in y]

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save the prepared data
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)


In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the prepared data
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

# Define the model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes: bench press, squat, deadlift
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Save the model
model.save('form_critique_model.h5')


Epoch 1/50


C:\Users\ramzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


384/384 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4526 - loss: 2.6912 - val_accuracy: 0.6400 - val_loss: 0.8487
Epoch 2/50
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5910 - loss: 0.8543 - val_accuracy: 0.7105 - val_loss: 0.6880
Epoch 3/50
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7595 - loss: 0.6102 - val_accuracy: 0.8869 - val_loss: 0.3326
Epoch 4/50
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8745 - loss: 0.3677 - val_accuracy: 0.9247 - val_loss: 0.2203
Epoch 5/50
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9135 - loss: 0.2657 - val_accuracy: 0.9508 - val_loss: 0.1851
Epoch 6/50
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9335 - loss: 0.2239 - val_accuracy: 0.9521 - val_loss: 0.1636
Epoch 7/50
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9387 - loss: 0.2018 - val_accuracy: 0.9628 - val_loss: 0.1390
Epoch 8/50
384/384 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9459 - loss: 0.1775 - val_accuracy: 0.9632 - val_

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('form_critique_model.h5')

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

label_to_exercise = {0: 'bench_press', 1: 'squat', 2: 'deadlift'}

def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
    return []

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0:
        angle = 360-angle
    return angle

def main():
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)  # Flip the frame horizontally

        keypoints = extract_keypoints(frame)
        if keypoints:
            # Calculate angles (example: elbow angle)
            angle = calculate_angle(keypoints[11], keypoints[13], keypoints[15])  # Example for left elbow
            input_data = np.array(keypoints).flatten().tolist() + [angle]
            input_data = np.array(input_data).reshape(1, -1)
            prediction = model.predict(input_data)
            exercise = label_to_exercise[np.argmax(prediction)]
            cv2.putText(frame, f'Exercise: {exercise}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow('Real-Time Exercise Classification', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

c:\Users\ramzi\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import pyttsx3
import time

# Load the trained model
model = load_model('form_critique_model.h5')

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

label_to_exercise = {0: 'bench_press', 1: 'squat', 2: 'deadlift'}

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Print all available voices for debugging
voices = engine.getProperty('voices')
for voice in voices:
    print("Voice:")
    print(" - ID:", voice.id)
    print(" - Name:", voice.name)
    print(" - Languages:", voice.languages)
    print(" - Gender:", voice.gender)
    print(" - Age:", voice.age)

# Choose the Zira voice if available
zira_voice_found = False
for voice in voices:
    if 'Hazel' in voice.name.lower():
        engine.setProperty('voice', voice.id)
        zira_voice_found = True
        break

# If Zira voice not found, choose the default voice
if not zira_voice_found:
    engine.setProperty('voice', voices[0].id)

def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
    return []

def normalize_keypoints(keypoints, image_width, image_height):
    normalized_keypoints = []
    for kp in keypoints:
        normalized_keypoints.append((kp[0] / image_width, kp[1] / image_height, kp[2]))
    return normalized_keypoints

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

def critique_form(exercise_type, keypoints):
    if exercise_type == 'bench_press':
        # Example rules for bench press form critique
        elbow_angle = calculate_angle(keypoints[11], keypoints[13], keypoints[15])  # Example for left elbow
        if elbow_angle > 90:
            return "Adjust your elbow position in bench press."
        # Add more rules as needed
    elif exercise_type == 'squat':
        # Example rules for squat form critique
        hip_knee_alignment = calculate_angle(keypoints[23], keypoints[25], keypoints[27])  # Example for hip-knee alignment
        if hip_knee_alignment > 10:
            return "Ensure your hips and knees are aligned properly in squat."
        # Add more rules as needed
    elif exercise_type == 'deadlift':
        # Example rules for deadlift form critique
        back_angle = calculate_angle(keypoints[11], keypoints[23], keypoints[25])  # Example for back angle
        if back_angle < 170:
            return "Maintain a straight back throughout the deadlift."
        # Add more rules as needed
    else:
        return "No critique rules defined for this exercise yet."
    
    return "Form looks good."

def speak_feedback(feedback):
    engine.say(feedback)
    engine.runAndWait()

def main():
    cap = cv2.VideoCapture(0)
    last_feedback_time = time.time()
    feedback_delay = 5.0  # Delay in seconds for speaking feedback

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)  # Use 1 for horizontal flip, 0 for vertical flip
        
        keypoints = extract_keypoints(frame)
        if keypoints:
            # Calculate angles (example: elbow angle)
            angle = calculate_angle(keypoints[11], keypoints[13], keypoints[15])  # Example for left elbow
            input_data = np.array(keypoints).flatten().tolist() + [angle]
            input_data = np.array(input_data).reshape(1, -1)
            prediction = model.predict(input_data)
            exercise = label_to_exercise[np.argmax(prediction)]
            feedback = critique_form(exercise, keypoints)
            cv2.putText(frame, feedback, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            current_time = time.time()
            if current_time - last_feedback_time >= feedback_delay:
                speak_feedback(feedback)
                last_feedback_time = current_time

        cv2.imshow('Real-Time Form Critique', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


Voice:
 - ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_FR-FR_HORTENSE_11.0
 - Name: Microsoft Hortense Desktop - French
 - Languages: []
 - Gender: None
 - Age: None
Voice:
 - ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0
 - Name: Microsoft David Desktop - English (United States)
 - Languages: []
 - Gender: None
 - Age: None
Voice:
 - ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
 - Name: Microsoft Zira Desktop - English (United States)
 - Languages: []
 - Gender: None
 - Age: None
Voice:
 - ID: HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-GB_HAZEL_11.0
 - Name: Microsoft Hazel Desktop - English (Great Britain)
 - Languages: []
 - Gender: None
 - Age: None
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import base64

from flask import Flask
from flask_socketio import SocketIO, emit

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

# Load the trained model
model = load_model('form_critique_model.h5')

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

label_to_exercise = {0: 'bench_press', 1: 'squat', 2: 'deadlift'}

def extract_keypoints(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks:
        return [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
    return []

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

def critique_form(exercise_type, keypoints):
    if exercise_type == 'bench_press':
        elbow_angle = calculate_angle(keypoints[11], keypoints[13], keypoints[15])  # Example for left elbow
        if elbow_angle > 90:
            return "Adjust your elbow position in bench press."
    elif exercise_type == 'squat':
        hip_knee_alignment = calculate_angle(keypoints[23], keypoints[25], keypoints[27])  # Example for hip-knee alignment
        if hip_knee_alignment > 10:
            return "Ensure your hips and knees are aligned properly in squat."
    elif exercise_type == 'deadlift':
        back_angle = calculate_angle(keypoints[11], keypoints[23], keypoints[25])  # Example for back angle
        if back_angle < 170:
            return "Maintain a straight back throughout the deadlift."
    else:
        return "No critique rules defined for this exercise yet."
    
    return "Form looks good."

@socketio.on('video_frame')
def handle_video_frame(data):
    frame = base64.b64decode(data['image'])
    np_arr = np.frombuffer(frame, np.uint8)
    img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
    
    keypoints = extract_keypoints(img)
    if keypoints:
        angle = calculate_angle(keypoints[11], keypoints[13], keypoints[15])  # Example for left elbow
        input_data = np.array(keypoints).flatten().tolist() + [angle]
        input_data = np.array(input_data).reshape(1, -1)
        prediction = model.predict(input_data)
        exercise = label_to_exercise[np.argmax(prediction)]
        feedback = critique_form(exercise, keypoints)
    else:
        feedback = "No keypoints detected."

    emit('form_critique', feedback)

if __name__ == '__main__':
    socketio.run(app, port=5000, allow_unsafe_werkzeug=True)
